In [ ]:
#cnn here is used

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from keras.models import Sequential
from keras.layers import Conv2D ,MaxPooling2D,Dense,Dropout,Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

ImportError: cannot import name 'ImageDataGenerator' from 'keras.preprocessing.image' (/usr/local/lib/python3.10/dist-packages/keras/api/preprocessing/image/__init__.py)

In [ ]:
train_data_gen=ImageDataGenerator(rescale=1./255)
validation_data_gen=ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator=train_data_gen.flow_from_directory('/content/drive/MyDrive/emotions/train',
                                                   target_size=(48,48),
                                                   batch_size=64,
                                                   color_mode="grayscale",
                                                   class_mode="categorical")


Found 28778 images belonging to 7 classes.


In [ ]:
from keras.backend import dropout
validation_generator=validation_data_gen.flow_from_directory(
    '/content/drive/MyDrive/emotions/test',
    target_size=(48,48),
    batch_size=64,
    color_mode='grayscale',
    class_mode="categorical")

Found 7188 images belonging to 7 classes.


In [ ]:
from tensorflow.keras.optimizers import Adam


In [ ]:
emotion_model=Sequential()
emotion_model.add(Conv2D(32,kernel_size=(3,3),input_shape=(48,48,1),activation='relu'))
emotion_model.add(Conv2D(64,(3,3),activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(32,(3,3),activation='relu'))
emotion_model.add(MaxPooling2D((2,2)))
emotion_model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024,activation='relu'))
emotion_model.add(Dropout(0.5))
#7=emotions
emotion_model.add(Dense(7,activation='softmax'))


emotion_model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(learning_rate=0.0001),
    metrics=['accuracy']
)



In [ ]:
#train
emotion_model_info=emotion_model.fit_generator(
    train_generator,
    steps_per_epoch=28709//64,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=7178//64
)

#save model structure in jasopn file
model_json=emotion_model.to_json()
with open("emotion_model.json","w") as json_file:
  json_file.write(model_json)
#save trained model weight in .h5 file
emotion_model.save_weights('emotion_mode.h5')




Epoch 1/10


<ipython-input-9-dda76cbec2d6>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info=emotion_model.fit_generator(


140/448 [========>.....................] - ETA: 1:31:53 - loss: 1.8289 - accuracy: 0.2507

In [ ]:
import  cv2
import numpy as np
from keras.models import model_from_json


In [ ]:
emotion_dict={0:"Angry",1:"Disgusted",2:"Fearful",3:"Happy",4:"Neutral",5:"Sad",6:"Suprised"}


In [ ]:
#oad json and create model
json_file=open('emotion_model.json','r')
loaded_model_json=json_file.read()
json_file.close()
emotion_model=model_from_json(loaded_model_json)
#load weights into new model
emotion_model.load_weights('/content/emotion_mode.h5')
print("loaded model from disk")

In [ ]:
#start the webcam
from google.colab.patches import cv2_imshow
#cap=cv2.VideoCapture(0)
cap=cv2.VideoCapture("emotions/samples/istockphoto-1152822473-640_adpp_is.mp4")
while True:
  ret,frame=cap.read()
  frame=cv2.resize(frame,(1280,720))
  if not ret:
    break
  face_detector=cv2.CascadeClassifier('/content/haarcascade_frontalface_default.xml')
  gray_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
  #detect fzces available on cam
  num_faces=face_detector.detectMultiScale(gray_frame,scaleFactor=1.3,minNeighbors=5)
  #take each face available on the camera and preprocess it
  for(x,y,w,h) in num_faces:
    cv2.rectangle(frame,(x,y-50),(x+w,y+h+10),(0,255,0),4)
    roi_gray_frame=gray_frame[y:y+h,x:x+w]
    cropped_img=np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame,(48,48)),-1),0)
    #predict the emotions
    emotion_prediction=emotion_model.predict(cropped_img)
    maxindex=int(np.argmax(emotion_prediction))
    cv2.putText(frame,emotion_dict[maxindex],(x+5,y-20),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)


  cv2_imshow(frame)
  if cv2.waitKey(1) & 0xFF ==ord('q'):
    break
cap.release()
cv2.destroyAllWindows()


